In [1]:
# libraries

from datetime import datetime
import os
import glob
import requests 
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# Web scrapping

In [2]:
# download data 
# =============

link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSc_2y5N0I67wDU38DjDh35IZSIS30rQf7_NYZhtYYGU1jJYT6_kDx4YpF-qw0LSlGsBYP8pqM_a1Pd/pubhtml#'
req = requests.get(link)
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
tbody = soup.find_all('tbody')[0]
body = tbody.find_all('tr')

# print(tbody)
# print(body)

In [4]:
head_row = [i.text for i in body[0].find_all('td')]
head_row

['Patient Number',
 'State Patient Number',
 'Date Announced',
 'Age Bracket',
 'Gender',
 'Detected City',
 'Detected District',
 'Detected State',
 'State_code',
 'Current Status',
 'Notes',
 'Contracted from which Patient (Suspected)',
 'Nationality',
 'Type of transmission',
 'Status Change Date',
 'Source_1',
 'Source_2',
 'Source_3',
 'Backup Notes',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [5]:
contents = []

for i in range(len(body)):
    contents.append([i.text for i in body[i].find_all('td')])

# Saving to Dataframe

In [6]:
p_df = pd.DataFrame(contents[2:len(contents)], columns=head_row)
p_df.head()

,Patient Number,State Patient Number,Date Announced,Age Bracket,Gender,Detected City,Detected District,Detected State,State_code,Current Status,...,Source_2,Source_3,Backup Notes,,,,,,,
0,1,KL-TS-P1,30/01/2020,20,F,Thrissur,Thrissur,Kerala,KL,Recovered,...,https://weather.com/en-IN/india/news/news/2020...,Student from Wuhan,,,,,,,,None
1,2,KL-AL-P1,02/02/2020,,,Alappuzha,Alappuzha,Kerala,KL,Recovered,...,https://weather.com/en-IN/india/news/news/2020...,,Student from Wuhan,,,,,,,
2,3,KL-KS-P1,03/02/2020,,,Kasaragod,Kasaragod,Kerala,KL,Recovered,...,https://twitter.com/ANI/status/122422148580539...,https://weather.com/en-IN/india/news/news/2020...,Student from Wuhan,,,,,,,
3,4,DL-P1,02/03/2020,45,M,East Delhi (Mayur Vihar),East Delhi,Delhi,DL,Recovered,...,https://economictimes.indiatimes.com/news/poli...,,Travel history to Italy and Austria,,,,,,,
4,5,TS-P1,02/03/2020,24,M,Hyderabad,Hyderabad,Telangana,TG,Recovered,...,https://www.indiatoday.in/india/story/coronavi...,https://www.thehindu.com/news/national/coronav...,"Travel history to Dubai, Singapore contact",,,,,,,


# Data cleaning and transformations

In [7]:
# shape of dataframe
# ==================

p_df.shape

(3066, 26)

In [8]:
# columns
# =======

p_df.columns

Index(['Patient Number', 'State Patient Number', 'Date Announced',
       'Age Bracket', 'Gender', 'Detected City', 'Detected District',
       'Detected State', 'State_code', 'Current Status', 'Notes',
       'Contracted from which Patient (Suspected)', 'Nationality',
       'Type of transmission', 'Status Change Date', 'Source_1', 'Source_2',
       'Source_3', 'Backup Notes', '', '', '', '', '', '', ''],
      dtype='object')

In [9]:
# selecting important columns only
# ================================

p_df = p_df[['Patient Number', 'State Patient Number', 'Date Announced',
       'Age Bracket', 'Gender', 'Detected City', 'Detected District',
       'Detected State', 'Current Status', 'Notes',
       'Contracted from which Patient (Suspected)', 'Nationality',
       'Status Change Date', 'Source_1', 'Source_2', 'Source_3',
       'Backup Notes']]

In [10]:
# looking for missing values 
# ==========================

p_df.isna().sum()

Patient Number                               0
State Patient Number                         0
Date Announced                               0
Age Bracket                                  0
Gender                                       0
Detected City                                0
Detected District                            0
Detected State                               0
Current Status                               0
Notes                                        0
Contracted from which Patient (Suspected)    0
Nationality                                  0
Status Change Date                           0
Source_1                                     0
Source_2                                     0
Source_3                                     0
Backup Notes                                 0
dtype: int64

In [11]:
# no. of empty strings in each column
# ===================================

print(p_df.shape)

for i in p_df.columns:
    print(i, '\t', p_df[p_df[i]==''].shape[0])

(3066, 17)
Patient Number 	 54
State Patient Number 	 2609
Date Announced 	 487
Age Bracket 	 2299
Gender 	 2147
Detected City 	 2254
Detected District 	 1166
Detected State 	 487
Current Status 	 487
Notes 	 811
Contracted from which Patient (Suspected) 	 2365
Nationality 	 2641
Status Change Date 	 490
Source_1 	 498
Source_2 	 2117
Source_3 	 2911
Backup Notes 	 2706


In [12]:
# replacing empty strings with np.nan
# ==================================-

print(p_df.shape)

p_df = p_df.replace(r'', np.nan, regex=True)
p_df.isna().sum()

(3066, 17)


Patient Number                                 54
State Patient Number                         2609
Date Announced                                487
Age Bracket                                  2299
Gender                                       2147
Detected City                                2254
Detected District                            1166
Detected State                                487
Current Status                                487
Notes                                         811
Contracted from which Patient (Suspected)    2365
Nationality                                  2641
Status Change Date                            490
Source_1                                      498
Source_2                                     2117
Source_3                                     2911
Backup Notes                                 2706
dtype: int64

In [13]:
# droping empty rows (row with just row number but without patient entry
# ======================================================================

p_df.dropna(subset=['Detected State'], inplace=True)
print(p_df.shape)
p_df.isna().sum()

(2579, 17)


Patient Number                                  0
State Patient Number                         2122
Date Announced                                  0
Age Bracket                                  1812
Gender                                       1660
Detected City                                1767
Detected District                             679
Detected State                                  0
Current Status                                  0
Notes                                         324
Contracted from which Patient (Suspected)    1878
Nationality                                  2154
Status Change Date                              3
Source_1                                       11
Source_2                                     1630
Source_3                                     2424
Backup Notes                                 2219
dtype: int64

In [14]:
p_df.columns

Index(['Patient Number', 'State Patient Number', 'Date Announced',
       'Age Bracket', 'Gender', 'Detected City', 'Detected District',
       'Detected State', 'Current Status', 'Notes',
       'Contracted from which Patient (Suspected)', 'Nationality',
       'Status Change Date', 'Source_1', 'Source_2', 'Source_3',
       'Backup Notes'],
      dtype='object')

In [15]:
# rename dateframe columns 
# ========================

p_df.columns = ['_'.join(col.lower().split()) for col in p_df.columns]
p_df.rename(columns = {'contracted_from_which_patient_(suspected)':'suspected_contacted_patient'}, inplace=True)
p_df.sample(5)

,patient_number,state_patient_number,date_announced,age_bracket,gender,detected_city,detected_district,detected_state,current_status,notes,suspected_contacted_patient,nationality,status_change_date,source_1,source_2,source_3,backup_notes
1769,1770,NaN,01/04/2020,45,M,Mettupalayam,Coimbatore,Tamil Nadu,Hospitalized,Attended Delhi Religious Conference,E0,NaN,01/04/2020,https://twitter.com/NHM_TN/status/124533236456...,NaN,NaN,NaN
2298,2299,NaN,02/04/2020,NaN,NaN,NaN,Mumbai,Maharashtra,Hospitalized,NaN,NaN,NaN,02/04/2020,https://twitter.com/ANI/status/124569351450080...,https://twitter.com/ANI/status/124569425319748...,NaN,NaN
2454,2455,NaN,02/04/2020,NaN,NaN,NaN,NaN,Delhi,Hospitalized,NaN,NaN,NaN,02/04/2020,https://twitter.com/ANI/status/124572782601742...,NaN,NaN,NaN
540,541,NaN,24/03/2020,NaN,NaN,Ahmednagar,Ahmadnagar,Maharashtra,Hospitalized,Details awaited,NaN,NaN,24/03/2020,https://twitter.com/PTI_News/status/1242403827...,NaN,NaN,NaN
925,926,NaN,28/03/2020,28-35,M,Hajin,Bandipore,Jammu and Kashmir,Hospitalized,Contact Transmission P 531,P531,NaN,28/03/2020,https://twitter.com/ANI/status/124383125547019...,NaN,NaN,NaN


In [16]:
# creating patient id column from patient number
# ===============================================

p_df['p_id'] = p_df['patient_number'].apply(lambda x : 'P'+str(x))
p_df.columns

Index(['patient_number', 'state_patient_number', 'date_announced',
       'age_bracket', 'gender', 'detected_city', 'detected_district',
       'detected_state', 'current_status', 'notes',
       'suspected_contacted_patient', 'nationality', 'status_change_date',
       'source_1', 'source_2', 'source_3', 'backup_notes', 'p_id'],
      dtype='object')

In [17]:
p_df = p_df[['patient_number', 'p_id', 'state_patient_number', 'date_announced',
       'age_bracket', 'gender', 'detected_city', 'detected_district',
       'detected_state', 'current_status', 'notes',
       'suspected_contacted_patient', 'nationality', 'status_change_date',
       'source_1', 'source_2', 'source_3', 'backup_notes']]

# Saving Data

In [18]:
p_df.sample(5)

,patient_number,p_id,state_patient_number,date_announced,age_bracket,gender,detected_city,detected_district,detected_state,current_status,notes,suspected_contacted_patient,nationality,status_change_date,source_1,source_2,source_3,backup_notes
84,85,P85,KL-TV-R3,13/03/2020,NaN,NaN,Varkala,Thiruvananthapuram,Kerala,Recovered,"Travelled from Italy, Italian national stayed ...",NaN,Italy,24/03/2020,https://english.manoramaonline.com/news/kerala...,https://www.thehindu.com/news/national/kerala/...,http://dhs.kerala.gov.in/wp-content/uploads/20...,Italian national stayed in a resort at Varkala
91,92,P92,NaN,14/03/2020,NaN,NaN,Ahmednagar,Ahmadnagar,Maharashtra,Hospitalized,Travelled from Dubai,NaN,India,14/03/2020,https://www.indiatoday.in/india/story/coronavi...,NaN,NaN,Returned from Dubai
742,743,P743,NaN,27/03/2020,NaN,NaN,NaN,Nagpur,Maharashtra,Hospitalized,Details Awaited,NaN,NaN,27/03/2020,https://twitter.com/ANI/status/124340517651538...,NaN,NaN,NaN
2216,2217,P2217,NaN,02/04/2020,NaN,NaN,NaN,Malappuram,Kerala,Hospitalized,NaN,NaN,NaN,02/04/2020,https://twitter.com/ANI/status/124569271298621...,http://dhs.kerala.gov.in/wp-content/uploads/20...,NaN,NaN
2410,2411,P2411,NaN,02/04/2020,NaN,NaN,NaN,NaN,Delhi,Hospitalized,NaN,NaN,NaN,02/04/2020,https://twitter.com/ANI/status/124572782601742...,NaN,NaN,NaN


In [19]:
p_df.to_csv('patients_data.csv', index=False)